<a href="https://colab.research.google.com/github/hemil19/Predict-The-Flight-Ticket-Price-Hackathon/blob/master/Predict-The-Flight-Ticket-Price-Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_excel('/content/Data_Train.xlsx')
test=pd.read_excel('/content/Test_set.xlsx')
train.shape,test.shape

((10683, 11), (2671, 10))

In [3]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
train.loc[9039,'Total_Stops']='2 stops'

In [5]:
train.shape

(10683, 11)

# Select duplicate rows except first occurrence based on all columns

In [6]:
duplicateRowsDF = train[train.duplicated()]
print("Total Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF.shape

Total Duplicate Rows except first occurrence based on all columns are :


(220, 11)

In [7]:
train.drop_duplicates(keep='first',inplace=True)

# Combining train and test for faster processing

In [8]:
combine=train.append(test)
combine.reset_index(inplace=True)
combine.drop('index',axis=1,inplace=True)
combine.shape

(13134, 11)

In [9]:
combine['Depart_Time_Hour'] = pd.to_datetime(combine.Dep_Time).dt.hour
combine['Depart_Time_Minutes'] = pd.to_datetime(combine.Dep_Time).dt.minute
combine['Arr_Time_Hour'] = pd.to_datetime(combine.Arrival_Time).dt.hour
combine['Arr_Time_Minutes'] = pd.to_datetime(combine.Arrival_Time).dt.minute
combine.drop(['Route','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [10]:
combine.head()

,Airline,Date_of_Journey,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,IndiGo,24/03/2019,Banglore,New Delhi,2h 50m,non-stop,No info,3897.0,22,20,1,10
1,Air India,1/05/2019,Kolkata,Banglore,7h 25m,2 stops,No info,7662.0,5,50,13,15
2,Jet Airways,9/06/2019,Delhi,Cochin,19h,2 stops,No info,13882.0,9,25,4,25
3,IndiGo,12/05/2019,Kolkata,Banglore,5h 25m,1 stop,No info,6218.0,18,5,23,30
4,IndiGo,01/03/2019,Banglore,New Delhi,4h 45m,1 stop,No info,13302.0,16,50,21,35


In [11]:
combine['Total_Stops']=combine['Total_Stops'].replace('non-stop','0 stops')

In [12]:
combine['Total_Stops'].isnull().sum()

0

In [13]:
combine['Total_Stops']=[int(i[0]) for i in combine['Total_Stops'].values]

In [14]:
combine.head()

,Airline,Date_of_Journey,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,IndiGo,24/03/2019,Banglore,New Delhi,2h 50m,0,No info,3897.0,22,20,1,10
1,Air India,1/05/2019,Kolkata,Banglore,7h 25m,2,No info,7662.0,5,50,13,15
2,Jet Airways,9/06/2019,Delhi,Cochin,19h,2,No info,13882.0,9,25,4,25
3,IndiGo,12/05/2019,Kolkata,Banglore,5h 25m,1,No info,6218.0,18,5,23,30
4,IndiGo,01/03/2019,Banglore,New Delhi,4h 45m,1,No info,13302.0,16,50,21,35


In [15]:
combine['Additional_Info']=combine['Additional_Info'].replace('No Info','No info')

In [16]:
combine['sourcedestination']=[(i,j) for i,j in zip(combine['Source'],combine['Destination'])]

In [17]:
combine.drop(['Source','Destination'],axis=1,inplace=True)

In [18]:
combine['Date_of_Journey']=pd.to_datetime(combine['Date_of_Journey'])

In [19]:
combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination
0,IndiGo,2019-03-24,2h 50m,0,No info,3897.0,22,20,1,10,"(Banglore, New Delhi)"
1,Air India,2019-01-05,7h 25m,2,No info,7662.0,5,50,13,15,"(Kolkata, Banglore)"
2,Jet Airways,2019-09-06,19h,2,No info,13882.0,9,25,4,25,"(Delhi, Cochin)"
3,IndiGo,2019-12-05,5h 25m,1,No info,6218.0,18,5,23,30,"(Kolkata, Banglore)"
4,IndiGo,2019-01-03,4h 45m,1,No info,13302.0,16,50,21,35,"(Banglore, New Delhi)"


In [20]:
combine.dtypes

Airline                        object
Date_of_Journey        datetime64[ns]
Duration                       object
Total_Stops                     int64
Additional_Info                object
Price                         float64
Depart_Time_Hour                int64
Depart_Time_Minutes             int64
Arr_Time_Hour                   int64
Arr_Time_Minutes                int64
sourcedestination              object
dtype: object

In [21]:
combine['Duration']=(pd.to_timedelta(combine['Duration']).dt.seconds // 60).astype(int)

In [22]:
combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination
0,IndiGo,2019-03-24,170,0,No info,3897.0,22,20,1,10,"(Banglore, New Delhi)"
1,Air India,2019-01-05,445,2,No info,7662.0,5,50,13,15,"(Kolkata, Banglore)"
2,Jet Airways,2019-09-06,1140,2,No info,13882.0,9,25,4,25,"(Delhi, Cochin)"
3,IndiGo,2019-12-05,325,1,No info,6218.0,18,5,23,30,"(Kolkata, Banglore)"
4,IndiGo,2019-01-03,285,1,No info,13302.0,16,50,21,35,"(Banglore, New Delhi)"


In [23]:
combine['sourcedestination'].value_counts()

(Delhi, Cochin)          5491
(Kolkata, Banglore)      3570
(Banglore, Delhi)        1582
(Banglore, New Delhi)    1152
(Mumbai, Hyderabad)       883
(Chennai, Kolkata)        456
Name: sourcedestination, dtype: int64

In [24]:
combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination
0,IndiGo,2019-03-24,170,0,No info,3897.0,22,20,1,10,"(Banglore, New Delhi)"
1,Air India,2019-01-05,445,2,No info,7662.0,5,50,13,15,"(Kolkata, Banglore)"
2,Jet Airways,2019-09-06,1140,2,No info,13882.0,9,25,4,25,"(Delhi, Cochin)"
3,IndiGo,2019-12-05,325,1,No info,6218.0,18,5,23,30,"(Kolkata, Banglore)"
4,IndiGo,2019-01-03,285,1,No info,13302.0,16,50,21,35,"(Banglore, New Delhi)"


In [25]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
combine['sourcedestination']=le.fit_transform(combine['sourcedestination'].astype(str))

In [26]:
combine.dtypes

Airline                        object
Date_of_Journey        datetime64[ns]
Duration                        int64
Total_Stops                     int64
Additional_Info                object
Price                         float64
Depart_Time_Hour                int64
Depart_Time_Minutes             int64
Arr_Time_Hour                   int64
Arr_Time_Minutes                int64
sourcedestination               int64
dtype: object

In [27]:
combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination
0,IndiGo,2019-03-24,170,0,No info,3897.0,22,20,1,10,1
1,Air India,2019-01-05,445,2,No info,7662.0,5,50,13,15,4
2,Jet Airways,2019-09-06,1140,2,No info,13882.0,9,25,4,25,3
3,IndiGo,2019-12-05,325,1,No info,6218.0,18,5,23,30,4
4,IndiGo,2019-01-03,285,1,No info,13302.0,16,50,21,35,1


In [28]:
combine['week'] = pd.to_datetime(combine['Date_of_Journey'], format='%y-%m-%d')
combine['Month'] = combine.week.dt.month
combine['Day'] = combine.week.dt.day
combine['Dayofweek'] = combine.week.dt.dayofweek

def get_weekend(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0
combine['IsWeekend'] = combine['week'].apply(get_weekend)

combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination,week,Month,Day,Dayofweek,IsWeekend
0,IndiGo,2019-03-24,170,0,No info,3897.0,22,20,1,10,1,2019-03-24,3,24,6,1
1,Air India,2019-01-05,445,2,No info,7662.0,5,50,13,15,4,2019-01-05,1,5,5,1
2,Jet Airways,2019-09-06,1140,2,No info,13882.0,9,25,4,25,3,2019-09-06,9,6,4,0
3,IndiGo,2019-12-05,325,1,No info,6218.0,18,5,23,30,4,2019-12-05,12,5,3,0
4,IndiGo,2019-01-03,285,1,No info,13302.0,16,50,21,35,1,2019-01-03,1,3,3,0


In [29]:
combine.drop(['Date_of_Journey','week'],axis=1,inplace=True)

In [30]:
combine["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)
combine["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)

In [31]:
combine=pd.get_dummies(combine)

In [32]:
X = combine[combine['Price'].isnull()!=True].drop(['Price'], axis=1)
y = combine[combine['Price'].isnull()!=True]['Price']

test = combine[combine['Price'].isnull()==True].drop(['Price'], axis=1)

X.shape, y.shape, test.shape

((10463, 25), (10463,), (2671, 25))

In [33]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold

# Checking with LGBMRegressor

In [34]:
err = []
y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print(i, "err_lgm: ",1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    err.append(1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 2.79541e+06	training's rmse: 1671.95	valid_1's l2: 3.65668e+06	valid_1's rmse: 1912.24
[400]	training's l2: 2.17838e+06	training's rmse: 1475.93	valid_1's l2: 3.22888e+06	valid_1's rmse: 1796.91
[600]	training's l2: 1.82254e+06	training's rmse: 1350.02	valid_1's l2: 3.00521e+06	valid_1's rmse: 1733.55
[800]	training's l2: 1.56891e+06	training's rmse: 1252.56	valid_1's l2: 2.81676e+06	valid_1's rmse: 1678.32
[1000]	training's l2: 1.39409e+06	training's rmse: 1180.71	valid_1's l2: 2.711e+06	valid_1's rmse: 1646.51
[1200]	training's l2: 1.25418e+06	training's rmse: 1119.9	valid_1's l2: 2.61314e+06	valid_1's rmse: 1616.52
[1400]	training's l2: 1.14883e+06	training's rmse: 1071.83	valid_1's l2: 2.58927e+06	valid_1's rmse: 1609.12
[1600]	training's l2: 1.06806e+06	training's rmse: 1033.47	valid_1's l2: 2.52948e+06	valid_1's rmse: 1590.44
[1800]	training's l2: 1.00265e+06	training's rmse: 1001.33	valid_1's l2:

In [35]:
np.mean(err,0)

0.9423146619326521

# Feature Importance as per LGBM Regressor model

In [36]:
all_feat = feature_importance_df[["Feature", 
                                      "importance"]].groupby("Feature").mean().sort_values(by="importance", 
                                                                                           ascending=False)
all_feat.reset_index(inplace=True)
important_feat = list(all_feat['Feature'])
all_feat

,Feature,importance
0,Duration,12449.133333
1,Depart_Time_Hour,6757.066667
2,Arr_Time_Hour,6380.333333
3,Depart_Time_Minutes,6146.200000
4,Day,5691.400000
5,Arr_Time_Minutes,5596.800000
6,Month,5105.600000
7,Dayofweek,4589.266667
8,sourcedestination,3426.866667
9,Total_Stops,2836.200000


# Removing features which are not important as per LGBMRegressor Model

In [37]:
k=all_feat[all_feat['importance']==0]['Feature'].values
for i in k:
  combine.drop(i,axis=1,inplace=True)

In [38]:
X = combine[combine['Price'].isnull()!=True].drop(['Price'], axis=1)
y = combine[combine['Price'].isnull()!=True]['Price']

test = combine[combine['Price'].isnull()==True].drop(['Price'], axis=1)

X.shape, y.shape, test.shape

((10463, 23), (10463,), (2671, 23))

In [39]:
err = []
y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.06,
                       n_estimators=6000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print(i, "err_lgm: ",1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    err.append(1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 2.61309e+06	training's rmse: 1616.51	valid_1's l2: 3.4911e+06	valid_1's rmse: 1868.45
[400]	training's l2: 1.98369e+06	training's rmse: 1408.43	valid_1's l2: 3.05951e+06	valid_1's rmse: 1749.14
[600]	training's l2: 1.66296e+06	training's rmse: 1289.56	valid_1's l2: 2.86757e+06	valid_1's rmse: 1693.39
[800]	training's l2: 1.43221e+06	training's rmse: 1196.75	valid_1's l2: 2.72833e+06	valid_1's rmse: 1651.77
[1000]	training's l2: 1.26679e+06	training's rmse: 1125.52	valid_1's l2: 2.63407e+06	valid_1's rmse: 1622.98
[1200]	training's l2: 1.14042e+06	training's rmse: 1067.91	valid_1's l2: 2.58618e+06	valid_1's rmse: 1608.16
[1400]	training's l2: 1.05267e+06	training's rmse: 1026	valid_1's l2: 2.55365e+06	valid_1's rmse: 1598.01
[1600]	training's l2: 974530	training's rmse: 987.183	valid_1's l2: 2.53819e+06	valid_1's rmse: 1593.17
[1800]	training's l2: 913660	training's rmse: 955.856	valid_1's l2: 2.50556e+0

In [42]:
np.mean(err,0)

0.942389921525278

In [43]:
err_xgb = []
y_pred_tot_xgb = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = XGBRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print(i, "err_xgb: ",1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    err_xgb.append(1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_xgb.append(pred_test)

[11:22:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:9688.83	validation_1-rmse:9695.7
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[200]	validation_0-rmse:1501.6	validation_1-rmse:1652.41
[400]	validation_0-rmse:1269.28	validation_1-rmse:1521.7
[600]	validation_0-rmse:1127.78	validation_1-rmse:1460.07
[800]	validation_0-rmse:1028.32	validation_1-rmse:1426.06
[1000]	validation_0-rmse:952.336	validation_1-rmse:1409.26
[1200]	validation_0-rmse:894.079	validation_1-rmse:1401.48
[1400]	validation_0-rmse:843.807	validation_1-rmse:1391.1
[1600]	validation_0-rmse:806.685	validation_1-rmse:1394.95
Stopping. Best iteration:
[1497]	validation_0-rmse:824.414	validation_1-rmse:1388.94

1 err_xgb:  0.9454407727434117
[11:22:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:line

In [45]:
np.mean(err_xgb,0)

0.9445884765791238

# XGBRegressor gives good prediction as compared to LGBMRegressor

In [46]:
submission=pd.read_excel('/content/Sample_submission.xlsx')

In [47]:
submission['Price']=np.mean(y_pred_tot_xgb,0)

In [48]:
submission['Price'].head()

0    14108.403320
1     4672.237793
2    12456.784180
3    10911.308594
4     3430.336670
Name: Price, dtype: float32

In [49]:
submission.to_csv('xgb_average.csv',index=False)